In [9]:
# 🏢 SYSTÈME D'EXTRACTION D'INFORMATIONS D'ENTREPRISE
# =======================================================
# Architecture : 3 agents spécialisés avec pattern manager
# Input : Nom d'entreprise
# Output : Données structurées complètes

import os
import asyncio
import json
import logging
from datetime import datetime
from typing import List, Optional
from pydantic import BaseModel, Field
from agents import Agent, Runner, function_tool, WebSearchTool
from dotenv import load_dotenv
load_dotenv()

True

In [10]:

# =======================================================
# 📊 MODÈLES DE DONNÉES STRUCTURÉES (CONFORMES À LA DOC OPENAI)
# =======================================================

class SubsidiaryInfo(BaseModel):
    """Modèle pour les informations détaillées d'une filiale"""
    subsidiary_name: str = Field(description="Nom de la filiale")
    subsidiary_address: str = Field(description="Adresse complète de la filiale")
    subsidiary_city: str = Field(description="Ville de la filiale")
    subsidiary_country: str = Field(description="Pays de la filiale")
    subsidiary_type: Optional[str] = Field(default=None, description="Type de filiale (bureau, usine, centre de recherche, etc.)")
    business_activity: Optional[str] = Field(default=None, description="Activité spécifique de cette filiale")
    employee_count: Optional[str] = Field(default=None, description="Nombre d'employés de cette filiale")
    establishment_date: Optional[str] = Field(default=None, description="Date de création de la filiale")
    parent_company: str = Field(description="Nom de la société mère")
    confidence_score: float = Field(description="Score de confiance pour cette filiale 0-1", ge=0, le=1)
    sources: List[str] = Field(default=[], description="Sources d'information pour cette filiale")

class CompanyAnalysisResult(BaseModel):
    """Résultat de l'analyse d'entreprise (principale ou filiale)"""
    is_subsidiary: bool = Field(description="True si l'entreprise recherchée est une filiale")
    original_company_name: str = Field(description="Nom de l'entreprise recherchée à l'origine")
    parent_company_name: Optional[str] = Field(default=None, description="Nom de l'entreprise principale si c'est une filiale")
    analysis_reasoning: str = Field(description="Explication de l'analyse (filiale ou entreprise principale)")
    confidence_score: float = Field(description="Score de confiance de l'analyse 0-1", ge=0, le=1)

class CompanyInfo(BaseModel):
    """Modèle principal des informations d'entreprise - utilisé par output_type"""
    company_name: str = Field(description="Nom officiel de l'entreprise")
    headquarters_address: str = Field(description="Adresse complète du siège social")
    headquarters_city: str = Field(description="Ville du siège social")
    headquarters_country: str = Field(description="Pays du siège social")
    parent_company: Optional[str] = Field(default=None, description="Société mère si applicable")
    subsidiaries: List[str] = Field(default=[], description="Liste des noms des filiales")
    subsidiaries_details: List[SubsidiaryInfo] = Field(default=[], description="Informations détaillées sur chaque filiale")
    core_business: str = Field(description="Activités principales de l'entreprise")
    industry_sector: str = Field(description="Secteur d'activité")
    revenue: Optional[str] = Field(default=None, description="Chiffre d'affaires annuel")
    employee_count: Optional[str] = Field(default=None, description="Nombre d'employés total")
    confidence_score: float = Field(description="Score de confiance 0-1", ge=0, le=1)
    sources: List[str] = Field(default=[], description="Sources d'information utilisées")
    extraction_date: str = Field(default_factory=lambda: datetime.now().isoformat())

class ValidationResult(BaseModel):
    """Modèle pour les résultats de validation - utilisé par output_type"""
    is_valid: bool = Field(description="Données valides ou non")
    confidence_score: float = Field(description="Score de confiance global", ge=0, le=1)
    completeness_score: float = Field(description="Score de complétude 0-1", ge=0, le=1)
    quality_score: float = Field(description="Score de qualité des sources 0-1", ge=0, le=1)
    validation_errors: List[str] = Field(default=[], description="Erreurs de validation détectées")
    warnings: List[str] = Field(default=[], description="Avertissements et recommandations")
    quality_sources_count: int = Field(description="Nombre de sources de qualité identifiées")
    total_sources: int = Field(description="Nombre total de sources")
    recommendations: List[str] = Field(default=[], description="Recommandations d'amélioration")


In [11]:
# =======================================================
# 🔧 OUTILS DE RECHERCHE ET EXTRACTION
# =======================================================


# Plus besoin de fonction fictive - on utilise directement WebSearchTool
# qui est un outil hébergé OpenAI pour la recherche web

@function_tool
async def validate_company_data(company_data: str) -> str:
    """
    Validation et vérification croisée des données d'entreprise
    ADAPTÉE au nouveau modèle CompanyInfo avec champs plats
    
    Args:
        company_data: Données d'entreprise au format JSON
    
    Returns:
        Résultat de validation au format JSON
    """
    try:
        data = json.loads(company_data)
        errors = []
        warnings = []
        
        # Validation des champs obligatoires (NOUVEAU MODÈLE AVEC FILIALES)
        required_fields = [
            "company_name", 
            "headquarters_address", 
            "headquarters_city", 
            "headquarters_country",
            "core_business", 
            "industry_sector"
        ]
        
        for field in required_fields:
            if not data.get(field) or data.get(field).strip() == "":
                errors.append(f"Champ obligatoire manquant ou vide: {field}")
        
        # Validation des champs optionnels mais importants
        optional_important = ["parent_company", "revenue", "employee_count"]
        for field in optional_important:
            if not data.get(field):
                warnings.append(f"Information manquante (recommandée): {field}")
        
        # Validation de la cohérence des données
        if data.get("headquarters_city") and data.get("headquarters_country"):
            # Vérifier la cohérence ville/pays (logique basique)
            city = data["headquarters_city"].lower()
            country = data["headquarters_country"].lower()
            
            # Quelques vérifications de cohérence
            if "paris" in city and "france" not in country and "french" not in country:
                warnings.append("Incohérence possible: Paris mentionné mais pays non français")
            elif "new york" in city and "usa" not in country and "united states" not in country:
                warnings.append("Incohérence possible: New York mentionné mais pays non américain")
        
        # Validation des filiales
        subsidiaries = data.get("subsidiaries", [])
        subsidiaries_details = data.get("subsidiaries_details", [])
        
        if not subsidiaries:
            warnings.append("Aucune filiale identifiée (peut être normal pour certaines entreprises)")
        else:
            # Vérifier la cohérence entre la liste des filiales et les détails
            if len(subsidiaries) != len(subsidiaries_details):
                warnings.append(f"Incohérence: {len(subsidiaries)} filiales listées mais {len(subsidiaries_details)} détails fournis")
            
            # Valider chaque filiale
            for i, subsidiary in enumerate(subsidiaries_details):
                if not subsidiary.get("subsidiary_name"):
                    errors.append(f"Filiale {i+1}: nom manquant")
                if not subsidiary.get("subsidiary_address"):
                    warnings.append(f"Filiale {subsidiary.get('subsidiary_name', f'#{i+1}')}: adresse manquante")
                if not subsidiary.get("subsidiary_city"):
                    warnings.append(f"Filiale {subsidiary.get('subsidiary_name', f'#{i+1}')}: ville manquante")
                if not subsidiary.get("subsidiary_country"):
                    warnings.append(f"Filiale {subsidiary.get('subsidiary_name', f'#{i+1}')}: pays manquant")
        
        # Score de complétude (ADAPTÉ AU NOUVEAU MODÈLE)
        total_required = len(required_fields)
        completed_required = sum(1 for field in required_fields if data.get(field) and data.get(field).strip())
        completeness_score = completed_required / total_required
        
        # Score de qualité des sources
        sources = data.get("sources", [])
        high_quality_sources = ["sec.gov", "company official", "bloomberg", "reuters", "yahoo finance", "marketwatch"]
        quality_sources_count = len([s for s in sources if any(hq in s.lower() for hq in high_quality_sources)])
        quality_score = quality_sources_count / max(len(sources), 1)
        
        # Score de confiance global
        confidence_score = (completeness_score * 0.7) + (quality_score * 0.3)
        
        # Validation finale
        is_valid = len(errors) == 0 and completeness_score >= 0.8
        
        validation_result = {
            "is_valid": is_valid,
            "confidence_score": round(confidence_score, 2),
            "completeness_score": round(completeness_score, 2),
            "quality_score": round(quality_score, 2),
            "validation_errors": errors,
            "warnings": warnings,
            "quality_sources_count": quality_sources_count,
            "total_sources": len(sources),
            "recommendations": [
                "Vérifier la cohérence des informations géographiques",
                "S'assurer que les sources sont fiables et récentes",
                "Compléter les informations financières si disponibles"
            ] if warnings else []
        }
        
        return json.dumps(validation_result, ensure_ascii=False, indent=2)
        
    except Exception as e:
        logging.error(f"Erreur validation: {e}")
        return json.dumps({
            "error": str(e), 
            "is_valid": False,
            "confidence_score": 0.0
        })

In [ ]:


# =======================================================
# 🤖 AGENTS SPÉCIALISÉS
# =======================================================

# Agent 0: Analyseur d'entreprise (NOUVEAU - pour détecter filiale vs entreprise principale)
company_analyzer = Agent(
    name="Company Analyzer",
    instructions="""Tu es un expert en analyse d'entreprises pour déterminer si une entreprise est une filiale ou une entreprise principale.

Ton rôle:
1. Analyser si l'entreprise recherchée est une filiale ou une entreprise principale
2. Si c'est une filiale, identifier l'entreprise mère
3. Fournir un raisonnement clair de l'analyse
4. Évaluer la confiance de l'analyse

Stratégie de recherche:
- Rechercher "{company_name} parent company" ou "{company_name} société mère"
- Rechercher "{company_name} subsidiary of" ou "{company_name} filiale de"
- Rechercher "{company_name} owned by" ou "{company_name} propriété de"
- Rechercher "{company_name} company structure"
- Rechercher "{company_name} corporate hierarchy"

Critères d'analyse:
- Si l'entreprise a une société mère clairement identifiée → C'est une filiale
- Si l'entreprise est indépendante et a des filiales → C'est une entreprise principale
- Si l'entreprise fait partie d'un groupe → Analyser la structure

Tu dois retourner un CompanyAnalysisResult structuré.""",
    
    tools=[
        WebSearchTool(), 
    ],
    output_type=CompanyAnalysisResult,
    model="gpt-5-nano"
)

# Agent 1: Extracteur d'informations (UTILISANT WebSearchTool)
information_extractor = Agent(
    name="Information Extractor",
    instructions="""Tu es un expert en extraction d'informations d'entreprise. 

Ton rôle:
1. Utiliser WebSearchTool pour rechercher des informations complètes sur l'entreprise
2. Extraire: siège social, maison mère, secteur d'activité, données financières
3. Maximiser la précision en utilisant des sources fiables
4. Citer toutes les sources utilisées

Stratégie de recherche:
- Rechercher "{company_name} headquarters" ou "{company_name} siège social"
- Rechercher "{company_name} company information"
- Rechercher "{company_name} annual report" pour les données financières
- Rechercher "{company_name} SEC filing" pour les entreprises américaines
- Rechercher "{company_name} industry sector"

Priorités:
- Précision des données
- Exhaustivité des informations
- Fiabilité des sources""",
    
    tools=[WebSearchTool()],
    model="gpt-5-nano"
)

# Agent 1.5: Extracteur spécialisé pour les filiales (UTILISANT WebSearchTool)
subsidiaries_extractor = Agent(
    name="Subsidiaries Extractor",
    instructions="""Tu es un expert en extraction d'informations sur les filiales d'entreprise.

Ton rôle:
1. Utiliser WebSearchTool pour rechercher toutes les filiales d'une entreprise
2. Extraire pour chaque filiale: nom, adresse complète, ville, pays, type, activité, employés
3. Identifier le type de filiale (siège régional, usine, centre de recherche, bureau commercial)
4. Vérifier la cohérence des informations géographiques
5. Évaluer la fiabilité des sources pour chaque filiale

Stratégie de recherche:
- Rechercher "{company_name} subsidiaries" ou "{company_name} filiales"
- Rechercher "{company_name} international offices"
- Rechercher "{company_name} global locations"
- Rechercher "{company_name} annual report" pour les informations officielles
- Rechercher "{company_name} SEC filing" pour les entreprises américaines

Tu dois retourner des données structurées au format SubsidiaryInfo pour chaque filiale identifiée.""",
    
    tools=[WebSearchTool()],  # Utilisation directe de WebSearchTool
    model="gpt-5-nano"
)

# Agent 2: Validateur de données (AMÉLIORÉ avec output_type)
data_validator = Agent(
    name="Data Validator",
    instructions="""Tu es un spécialiste de la validation de données d'entreprise.

Ton rôle:
1. Analyser la cohérence des données extraites
2. Vérifier la complétude des informations selon le nouveau modèle CompanyInfo
3. Identifier les incohérences ou données manquantes
4. Calculer un score de confiance global

Critères de validation (NOUVEAU MODÈLE):
- Champs obligatoires: company_name, headquarters_address, headquarters_city, headquarters_country, core_business, industry_sector
- Champs optionnels importants: parent_company, revenue, employee_count
- Cohérence géographique (ville/pays)
- Qualité et fiabilité des sources
- Logique des relations parent-filiale

Tu dois retourner un ValidationResult structuré avec:
- is_valid (bool)
- confidence_score (float 0-1)
- completeness_score (float 0-1) 
- quality_score (float 0-1)
- validation_errors (liste des erreurs)
- warnings (liste des avertissements)
- recommendations (liste des recommandations)""",
    
    tools=[validate_company_data],
    output_type=ValidationResult,  # Sortie structurée pour la validation
    model="gpt-5-nano"
)

# Agent 3: Manager/Orchestrateur (WORKFLOW INTELLIGENT FILIALE ↔ ENTREPRISE PRINCIPALE)
extraction_manager = Agent(
    name="Extraction Manager",
    instructions="""Tu es le chef d'orchestre de l'extraction d'informations d'entreprise avec stratégie intelligente.

WORKFLOW STRATÉGIQUE:
1. ANALYSER l'entreprise recherchée (filiale ou entreprise principale ?)
2. Si FILIALE → Extraire l'entreprise principale et TOUTES ses filiales
3. Si ENTREPRISE PRINCIPALE → Extraire directement ses informations et filiales
4. VALIDER et structurer le résultat final

STRATÉGIE DE RECHERCHE:
- D'abord analyser si "{company_name}" est une filiale ou entreprise principale
- Si c'est une filiale, rechercher l'entreprise mère et extraire TOUTES ses filiales
- Si c'est une entreprise principale, extraire directement ses informations
- Toujours fournir le contexte complet (entreprise principale + toutes ses filiales)

IMPORTANT: Tu dois retourner des données structurées au format CompanyInfo avec EXACTEMENT ces champs:
- company_name (string) - TOUJOURS l'entreprise principale
- headquarters_address (string)
- headquarters_city (string) 
- headquarters_country (string)
- parent_company (string ou null)
- subsidiaries (liste de noms des filiales)
- subsidiaries_details (liste d'objets SubsidiaryInfo avec adresses et détails)
- core_business (string)
- industry_sector (string)
- revenue (string ou null)
- employee_count (string ou null)
- confidence_score (float entre 0 et 1)
- sources (liste de strings)
- extraction_date (string ISO format)

NOUVEAU: Chaque filiale dans subsidiaries_details doit contenir:
- subsidiary_name, subsidiary_address, subsidiary_city, subsidiary_country
- subsidiary_type, business_activity, employee_count, establishment_date
- parent_company, confidence_score, sources

Si les données sont incomplètes ou peu fiables, relance les recherches.""",
    
    tools=[
        company_analyzer.as_tool(
            tool_name="analyze_company_type",
            tool_description="Analyser si l'entreprise est une filiale ou une entreprise principale"
        ),
        information_extractor.as_tool(
            tool_name="extract_company_information",
            tool_description="Extraire les informations complètes d'une entreprise"
        ),
        subsidiaries_extractor.as_tool(
            tool_name="extract_subsidiaries_details",
            tool_description="Extraire les informations détaillées sur toutes les filiales"
        ),
        data_validator.as_tool(
            tool_name="validate_extracted_data",
            tool_description="Valider et scorer les données extraites"
        )
    ],
    output_type=CompanyInfo,  # Sortie structurée garantie avec le NOUVEAU modèle
    model="gpt-4o"
)

print("✅ Agent extraction_manager recréé avec le nouveau modèle CompanyInfo")

✅ Agent extraction_manager recréé avec le nouveau modèle CompanyInfo


In [13]:
# =======================================================
# 🎯 PIPELINE PRINCIPAL D'EXTRACTION
# =======================================================

class EnterpriseExtractionPipeline:
    """Pipeline principal pour l'extraction d'informations d'entreprise"""
    
    def __init__(self, openai_api_key: str):
        """
        Initialise le pipeline d'extraction
        
        Args:
            openai_api_key: Clé API OpenAI
        """
        self.api_key = openai_api_key
        self.setup_logging()
        
    def setup_logging(self):
        """Configure le logging"""
        logging.basicConfig(
            level=logging.INFO,
            format='%(asctime)s - %(levelname)s - %(message)s',
            handlers=[
                logging.FileHandler('enterprise_extraction.log'),
                logging.StreamHandler()
            ]
        )
        self.logger = logging.getLogger(__name__)
    
    async def extract_company_information(self, company_name: str) -> CompanyInfo:
        """
        Extrait les informations complètes d'une entreprise
        
        Args:
            company_name: Nom de l'entreprise à analyser
            
        Returns:
            CompanyInfo: Données structurées de l'entreprise
        """
        self.logger.info(f"🔍 Début extraction pour: {company_name}")
        
        try:
            # Créer le runner
            runner = Runner()
            
            # Lancer l'extraction via le manager
            result = await runner.run(
                extraction_manager,
                f"Extraire toutes les informations d'entreprise pour: {company_name}"
            )
            
            # Extraire les données structurées (conforme à la doc OpenAI Agents)
            company_info = result.final_output
            
            # Vérifier que le résultat est bien structuré
            if not isinstance(company_info, CompanyInfo):
                self.logger.warning(f"⚠️ Résultat non structuré pour {company_name}, conversion en cours...")
                # Si le résultat n'est pas structuré, essayer de le convertir
                if hasattr(company_info, 'model_dump'):
                    company_info = CompanyInfo(**company_info.model_dump())
                else:
                    raise ValueError("Résultat non structuré et non convertible")
            
            # Vérifier et corriger les attributs manquants
            if not hasattr(company_info, 'subsidiaries_details'):
                self.logger.warning(f"⚠️ Attribut subsidiaries_details manquant pour {company_name}, ajout d'une liste vide...")
                # Créer un nouvel objet CompanyInfo avec tous les attributs
                company_info = CompanyInfo(
                    company_name=company_info.company_name,
                    headquarters_address=company_info.headquarters_address,
                    headquarters_city=company_info.headquarters_city,
                    headquarters_country=company_info.headquarters_country,
                    parent_company=getattr(company_info, 'parent_company', None),
                    subsidiaries=getattr(company_info, 'subsidiaries', []),
                    subsidiaries_details=[],  # Liste vide par défaut
                    core_business=company_info.core_business,
                    industry_sector=company_info.industry_sector,
                    revenue=getattr(company_info, 'revenue', None),
                    employee_count=getattr(company_info, 'employee_count', None),
                    confidence_score=company_info.confidence_score,
                    sources=company_info.sources,
                    extraction_date=getattr(company_info, 'extraction_date', datetime.now().isoformat())
                )
            
            self.logger.info(f"✅ Extraction terminée pour {company_name}")
            self.logger.info(f"📊 Score de confiance: {company_info.confidence_score:.2f}")
            self.logger.info(f"🏢 Siège: {company_info.headquarters_city}, {company_info.headquarters_country}")
            self.logger.info(f"🏢 Filiales: {len(company_info.subsidiaries)} listées, {len(company_info.subsidiaries_details)} détaillées")
            
            return company_info
            
        except Exception as e:
            self.logger.error(f"❌ Erreur extraction pour {company_name}: {e}")
            # Retour avec données minimales en cas d'erreur
            return CompanyInfo(
                company_name=company_name,
                headquarters_address="Non disponible",
                headquarters_city="Non disponible",
                headquarters_country="Non disponible",
                parent_company=None,
                subsidiaries=[],
                subsidiaries_details=[],
                core_business="Non disponible",
                industry_sector="Non disponible",
                revenue=None,
                employee_count=None,
                confidence_score=0.0,
                sources=["error"],
                extraction_date=datetime.now().isoformat()
            )
    
    async def batch_extract(self, company_names: List[str]) -> List[CompanyInfo]:
        """
        Extraction en lot pour plusieurs entreprises
        
        Args:
            company_names: Liste des noms d'entreprises
            
        Returns:
            Liste des informations d'entreprise
        """
        self.logger.info(f"🔄 Extraction en lot pour {len(company_names)} entreprises")
        
        # Traitement parallèle avec limite de concurrence
        semaphore = asyncio.Semaphore(3)  # Max 3 extractions simultanées
        
        async def extract_with_semaphore(company_name):
            async with semaphore:
                return await self.extract_company_information(company_name)
        
        tasks = [extract_with_semaphore(name) for name in company_names]
        results = await asyncio.gather(*tasks, return_exceptions=True)
        
        # Filtrer les erreurs
        valid_results = [r for r in results if isinstance(r, CompanyInfo)]
        
        self.logger.info(f"✅ Extraction en lot terminée: {len(valid_results)}/{len(company_names)} réussies")
        return valid_results
    
    def export_to_json(self, company_info: CompanyInfo, filename: Optional[str] = None) -> str:
        """
        Exporte les données vers un fichier JSON
        
        Args:
            company_info: Données d'entreprise
            filename: Nom du fichier (optionnel)
            
        Returns:
            Chemin du fichier créé
        """
        if not filename:
            safe_name = "".join(c for c in company_info.company_name if c.isalnum() or c in (' ', '-', '_')).rstrip()
            filename = f"extract_{safe_name}_{datetime.now().strftime('%Y%m%d_%H%M%S')}.json"
        
        with open(filename, 'w', encoding='utf-8') as f:
            json.dump(company_info.model_dump(), f, ensure_ascii=False, indent=2)
        
        self.logger.info(f"📄 Données exportées vers: {filename}")
        return filename


In [14]:
# =======================================================
# 📚 NOUVELLE STRATÉGIE INTELLIGENTE: FILIALE ↔ ENTREPRISE PRINCIPALE
# =======================================================

print("✅ NOUVELLE STRATÉGIE INTELLIGENTE: FILIALE ↔ ENTREPRISE PRINCIPALE")
print()
print("🧠 WORKFLOW INTELLIGENT:")
print("   1️⃣ ANALYSER: L'entreprise recherchée est-elle une filiale ou une entreprise principale ?")
print("   2️⃣ Si FILIALE → Remonter à l'entreprise mère et extraire TOUTES ses filiales")
print("   3️⃣ Si ENTREPRISE PRINCIPALE → Extraire directement ses informations et filiales")
print("   4️⃣ RÉSULTAT: Contexte complet (entreprise principale + toutes ses filiales)")
print()
print("🎯 AVANTAGES DE CETTE STRATÉGIE:")
print("   ✅ Recherche 'Beats Electronics' → Retourne Apple + toutes les filiales d'Apple")
print("   ✅ Recherche 'Apple Inc.' → Retourne Apple + toutes ses filiales")
print("   ✅ Contexte complet dans tous les cas")
print("   ✅ Pas de confusion entre filiale et entreprise principale")
print("   ✅ Extraction exhaustive des filiales liées")
print()
print("🔍 AGENTS SPÉCIALISÉS:")
print("   🤖 Company Analyzer: Détecte filiale vs entreprise principale")
print("   🤖 Information Extractor: Extrait les infos de l'entreprise principale")
print("   🤖 Subsidiaries Extractor: Extrait TOUTES les filiales liées")
print("   🤖 Data Validator: Valide la cohérence des données")
print()
print("🚀 RÉSULTAT: Système intelligent qui comprend la structure d'entreprise!")


✅ NOUVELLE STRATÉGIE INTELLIGENTE: FILIALE ↔ ENTREPRISE PRINCIPALE

🧠 WORKFLOW INTELLIGENT:
   1️⃣ ANALYSER: L'entreprise recherchée est-elle une filiale ou une entreprise principale ?
   2️⃣ Si FILIALE → Remonter à l'entreprise mère et extraire TOUTES ses filiales
   3️⃣ Si ENTREPRISE PRINCIPALE → Extraire directement ses informations et filiales
   4️⃣ RÉSULTAT: Contexte complet (entreprise principale + toutes ses filiales)

🎯 AVANTAGES DE CETTE STRATÉGIE:
   ✅ Recherche 'Beats Electronics' → Retourne Apple + toutes les filiales d'Apple
   ✅ Recherche 'Apple Inc.' → Retourne Apple + toutes ses filiales
   ✅ Contexte complet dans tous les cas
   ✅ Pas de confusion entre filiale et entreprise principale
   ✅ Extraction exhaustive des filiales liées

🔍 AGENTS SPÉCIALISÉS:
   🤖 Company Analyzer: Détecte filiale vs entreprise principale
   🤖 Information Extractor: Extrait les infos de l'entreprise principale
   🤖 Subsidiaries Extractor: Extrait TOUTES les filiales liées
   🤖 Data Validato

In [15]:
# =======================================================
# 🚀 FONCTION PRINCIPALE D'EXTRACTION D'ENTREPRISE
# =======================================================

async def extract_company_data(company_name: str) -> dict:
    """
    Fonction principale pour extraire les données d'entreprise
    
    Args:
        company_name (str): Nom de l'entreprise à analyser
        
    Returns:
        dict: Données d'entreprise formatées en JSON
    """
    try:
        print(f"🔍 Début de l'extraction pour: {company_name}")
        
        # Créer le runner
        runner = Runner()
        
        # Exécuter les agents avec la stratégie intelligente
        result = await runner.run(
            extraction_manager,
            f"Extraire toutes les informations d'entreprise pour: {company_name}"
        )
        
        # Récupérer le résultat structuré
        company_info = result.final_output
        
        # Vérifier et corriger les attributs manquants
        if not hasattr(company_info, 'subsidiaries_details'):
            company_info = CompanyInfo(
                company_name=company_info.company_name,
                headquarters_address=company_info.headquarters_address,
                headquarters_city=company_info.headquarters_city,
                headquarters_country=company_info.headquarters_country,
                parent_company=getattr(company_info, 'parent_company', None),
                subsidiaries=getattr(company_info, 'subsidiaries', []),
                subsidiaries_details=[],
                core_business=company_info.core_business,
                industry_sector=company_info.industry_sector,
                revenue=getattr(company_info, 'revenue', None),
                employee_count=getattr(company_info, 'employee_count', None),
                confidence_score=company_info.confidence_score,
                sources=company_info.sources,
                extraction_date=getattr(company_info, 'extraction_date', datetime.now().isoformat())
            )
        
        # Convertir en dictionnaire pour le JSON
        result_dict = {
            "company_name": company_info.company_name,
            "headquarters_address": company_info.headquarters_address,
            "headquarters_city": company_info.headquarters_city,
            "headquarters_country": company_info.headquarters_country,
            "parent_company": company_info.parent_company,
            "subsidiaries": company_info.subsidiaries,
            "subsidiaries_details": [
                {
                    "subsidiary_name": sub.subsidiary_name,
                    "subsidiary_address": sub.subsidiary_address,
                    "subsidiary_city": sub.subsidiary_city,
                    "subsidiary_country": sub.subsidiary_country,
                    "subsidiary_type": sub.subsidiary_type,
                    "business_activity": sub.business_activity,
                    "employee_count": sub.employee_count,
                    "establishment_date": sub.establishment_date,
                    "parent_company": sub.parent_company,
                    "confidence_score": sub.confidence_score,
                    "sources": sub.sources
                }
                for sub in company_info.subsidiaries_details
            ],
            "core_business": company_info.core_business,
            "industry_sector": company_info.industry_sector,
            "revenue": company_info.revenue,
            "employee_count": company_info.employee_count,
            "confidence_score": company_info.confidence_score,
            "sources": company_info.sources,
            "extraction_date": company_info.extraction_date,
            "extraction_status": "success",
            "total_subsidiaries": len(company_info.subsidiaries),
            "detailed_subsidiaries": len(company_info.subsidiaries_details)
        }
        
        print(f"✅ Extraction terminée pour: {company_info.company_name}")
        print(f"📊 Score de confiance: {company_info.confidence_score:.2f}")
        print(f"🏢 Filiales: {len(company_info.subsidiaries)} listées, {len(company_info.subsidiaries_details)} détaillées")
        
        return result_dict
        
    except Exception as e:
        print(f"❌ Erreur lors de l'extraction pour {company_name}: {e}")
        
        # Retourner un résultat d'erreur en JSON
        return {
            "company_name": company_name,
            "headquarters_address": "Non disponible",
            "headquarters_city": "Non disponible",
            "headquarters_country": "Non disponible",
            "parent_company": None,
            "subsidiaries": [],
            "subsidiaries_details": [],
            "core_business": "Non disponible",
            "industry_sector": "Non disponible",
            "revenue": None,
            "employee_count": None,
            "confidence_score": 0.0,
            "sources": ["error"],
            "extraction_date": datetime.now().isoformat(),
            "extraction_status": "error",
            "error_message": str(e),
            "total_subsidiaries": 0,
            "detailed_subsidiaries": 0
        }




In [16]:
# =======================================================
# 🧪 TEST DE LA FONCTION
# =======================================================

# Test de la fonction
async def test_extract_function(company_name: str):
    """Test de la fonction extract_company_data"""
    print("🚀 Test de la fonction extract_company_data...")
    
    # Test avec une entreprise
    result = await extract_company_data(company_name)
    
    # Afficher le résultat JSON
    print("\n📋 RÉSULTAT JSON:")
    print(json.dumps(result, indent=2, ensure_ascii=False))
    
    return result

# Lancer le test
company_name = "axxair"
test_result = await test_extract_function(company_name)

🚀 Test de la fonction extract_company_data...
🔍 Début de l'extraction pour: axxair
✅ Extraction terminée pour: S.F.E Group
📊 Score de confiance: 0.90
🏢 Filiales: 17 listées, 17 détaillées

📋 RÉSULTAT JSON:
{
  "company_name": "S.F.E Group",
  "headquarters_address": "4433 South Drive, Houston, TX 77053, USA",
  "headquarters_city": "Houston",
  "headquarters_country": "United States",
  "parent_company": null,
  "subsidiaries": [
    "S.F.E. Group Global Head Office – United States",
    "S.F.E. Group International Head Office – France",
    "Asia Pacific Regional Office",
    "China Regional Office",
    "Germany Regional Office",
    "India Regional Office",
    "Kingdom of Saudi Arabia Regional Office",
    "Latin America Regional Office",
    "South Korea Regional Office",
    "United Arab Emirates Regional Office",
    "United Kingdom Regional Office",
    "Vietnam Regional Office",
    "USA Regional Office – Wooster",
    "USA Regional Office – Connecticut",
    "USA Regional Off